# Construct a Trie from a Collection of Patterns

In [0]:
import numpy as np
from collections import defaultdict

In [0]:
class Trie:
  def __init__(self):
    self.next = np.full((10000,4),-1, dtype=int)
    self.node_idx = 0
    self.end = np.zeros(10000, dtype=int)
  def insert(self, pattern):
    edges = []
    mp = {
        'A' : 0,
        'C' : 1,
        'G' : 2,
        'T' : 3
    }
    root = 0
    for ch in pattern:
      if self.next[root][mp[ch]] == -1:
        self.node_idx += 1
        edges.append((root,self.node_idx,ch))
        self.next[root][mp[ch]] = self.node_idx
      root = self.next[root][mp[ch]]
    self.end[root] = 1
      #endif
    #endfor
    return edges
  def search(self, text):
    mp = {
        'A' : 0,
        'C' : 1,
        'G' : 2,
        'T' : 3
    }
    ans = []
    for i in range(0, len(text)):
      root = 0
      for j in range(i, len(text)):
        if self.next[root][mp[text[j]]] == -1:
          break
        else:
          root = self.next[root][mp[text[j]]]
          if self.end[root] == 1:
            ans.append(i)
            break
          #endif
        #endif
      #endfor
    #endfor
    return ans

In [0]:
def build_trie(patterns):
  trie = Trie()
  ans = []
  for pattern in patterns:
    ans.extend(trie.insert(pattern))
  #endfor
  return ans

In [0]:
edges = build_trie(['ATAGA','ATC','GAT'])
for u,v,node in edges:
  print(str(u) + '->' + str(v) + ':' + node)

0->1:A
1->2:T
2->3:A
3->4:G
4->5:A
2->6:C
0->7:G
7->8:A
8->9:T


#Implement TrieMatching

In [0]:
def trie_matching(text, patterns):
  trie = Trie()
  for pattern in patterns:
    trie.insert(pattern)
  #endfor
  return trie.search(text)

In [0]:
print(*trie_matching('AATCGGGTTCAATCGGGGT',['ATCG','GGGT']))

1 4 11 15


#Construct the Suffix Array of a String

In [0]:
def construct_suffix(text):
  suffix = [(text[idx::],idx) for idx in range(len(text))]
  suffix_after_sort = sorted(suffix, key=lambda x: x[0])
  return [idx for s,idx in suffix_after_sort]

In [0]:
ans = construct_suffix('AACGATAGCGGTAGA$')
print(', '.join([str(idx) for idx in ans]))

15, 14, 0, 1, 12, 6, 4, 2, 8, 13, 3, 7, 9, 10, 11, 5


#Pattern Matching with the Suffix Array

In [0]:
def find_pos(text, suffix_array, pattern):
  lo = 0
  hi = len(suffix_array) - 1
  ans = -1
  while lo<=hi:
    mid = lo + (hi-lo)//2
    suffix = text[suffix_array[mid]::][0:len(pattern)]
    if suffix == pattern:
      ans = mid
    if suffix < pattern:
      lo = mid + 1
    else:
      hi = mid - 1
  #endwhile
  if ans == -1:
    return []
  min_idx = ans
  lo = ans
  hi = len(suffix_array) - 1
  while lo <= hi:
    mid = lo + (hi-lo)//2
    suffix = text[suffix_array[mid]::][0:len(pattern)]
    if suffix == pattern:
      ans = mid
    if suffix > pattern:
      hi = mid - 1
    else:
      lo = mid + 1
  #endwhile
  return [suffix_array[idx] for idx in range(min_idx, ans+1)]

In [0]:
def pattern_match_with_suffix_array(text, patterns):
  suffix_array = construct_suffix(text)
  pos = []
  for pattern in patterns:
    pos.extend(find_pos(text, suffix_array, pattern))
  #endfor
  return sorted(pos)

In [0]:
ans = pattern_match_with_suffix_array('AATCGGGTTCAATCGGGGT', ['ATCG','GGGT'])
print(*ans)


1 4 11 15


# Construct the Burrows-Wheeler Transform of a String

In [0]:
def BWT(text):
  circular = [text[idx::] + text[0:idx:] for idx in range(len(text))]
  return ''.join([s[-1] for s in sorted(circular)])

In [0]:
print(BWT('GCGTGCCTGGTCA$'))

ACTGGCT$TGCGGC


#Generate the Last-to-First Mapping of a String

In [0]:
def generate_two_way_map(text):
  last = defaultdict(int)
  l_mp = {}
  for idx, ch in enumerate(text):
    last[ch] +=1
    l_mp[idx] = (ch,last[ch])
  #endfor
  first = defaultdict(int)
  f_mp = {}
  for idx, ch in enumerate(sorted(text)):
    first[ch] += 1
    f_mp[(ch,first[ch])] = idx
  #endfor
  return f_mp,l_mp 

In [0]:
def last_to_first(text, pos):
  f_mp,l_mp = generate_two_way_map(text)
  ch,no = l_mp[pos]
  print(f_mp[(ch,no)])

In [0]:
last_to_first('T$GACCA',3)

1


#Reconstruct a String from its Burrows-Wheeler Transform 

In [0]:
def inverse_BWT(text):
  genome = '$'
  idx = 0
  F,L = generate_two_way_map(text)
  ch , no = '$',1
  print()
  while True:
    if len(genome) == len(text):
      break
    #endif
    idx = F[(ch, no)]
    ch,no = L[idx]
    genome += ch
  #endwhile
  return genome[::-1]

In [0]:
print(inverse_BWT('TTCCTAACG$A'))


TACATCACGT$


#Implement BWMatching

In [0]:
def BW_match(text, patterns):
  original = inverse_BWT(text)
  suffix = sorted([original[idx::] for idx in range(len(original))])
  result = []
  for pattern in patterns:
    cnt = 0
    for s in suffix:
      if s.startswith(pattern):
        cnt += 1
      elif s > pattern:
        break
    #endfor
    result.append(cnt)
  return result

In [0]:
ans  = BW_match('TCCTCTATGAGATCCTATTCTATGAAACCTTCA$GACCAAAATTCTCCGGC', ['CCT','CAC','GAG','CAG','ATC'])
print(*ans)


2 1 1 0 1
